# Build the Neural Network

In [2]:
import os 
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


## Define the class

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [22]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9], device='cuda:0')


## Model Layers

In [26]:
# 3 images of the size 28x28
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten

`nn.Flatten` to convert each 28x28 image into a contiguos array of 784 pixel values (flatmap)

In [27]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


## nn.Linear

The linear layer is a module that applies a linear transformation on the input using its stored weights and biases.

In [32]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU

Non-linear activations are what create the complex mappings between the model's inputs and outputs. They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.

In this model, we use `nn.ReLU` between our linear layers, but there's other activations to introduce non-linearity in your model.

In [34]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.3601, -0.2100,  0.3283, -0.6649, -0.2401,  0.0648, -0.3847,  0.0588,
         -0.3499,  0.0461,  0.2230, -0.3372, -0.4617, -0.3646, -0.0540,  0.0462,
          0.0871,  0.0608,  0.2145,  0.1211],
        [-0.0192, -0.2929,  0.3272, -0.1555,  0.1211, -0.2932, -0.6418,  0.1952,
         -0.3219,  0.4129,  0.3528,  0.1417, -0.4630, -0.0921,  0.2602,  0.0009,
         -0.0431,  0.0473,  0.2303,  0.1406],
        [-0.0297, -0.2489,  0.2133, -0.7154, -0.2158,  0.0546, -0.6376, -0.1990,
         -0.4245,  0.1669,  0.5346, -0.1999, -0.3790,  0.1016,  0.1274,  0.3869,
          0.0137, -0.0250,  0.1615, -0.0995]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.3601, 0.0000, 0.3283, 0.0000, 0.0000, 0.0648, 0.0000, 0.0588, 0.0000,
         0.0461, 0.2230, 0.0000, 0.0000, 0.0000, 0.0000, 0.0462, 0.0871, 0.0608,
         0.2145, 0.1211],
        [0.0000, 0.0000, 0.3272, 0.0000, 0.1211, 0.0000, 0.0000, 0.1952, 0.0000,
         0.4129, 0.3528, 0.1417, 0.0000, 0.0000, 0.260

### nn.Sequential

`nn.Sequential` is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like `seq.modules`.

In [35]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

### nn.Softmax

The last linear layer of the neural network returns logits - raw values in [-infinity, infinity] - which are passed to the `nn.Softmax` module. The logits are scaled to values [0, 1] representing the model's predicted probabilities for each class. `dim` parameters indicates the dimension along which the values must sum to 1.

In [38]:
softmax = nn.Softmax(dim=1)
pred_probability = softmax(logits)

## Model Parameters

Many layers inside a neural network are parameterized, e.g. have associated weights and biases that are optimized during training. Subclassing `nn.Module` automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model's `parameters()` or `named_parameters()` methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.

In [39]:
print("Model strucutre: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model strucutre:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0336, -0.0165, -0.0338,  ..., -0.0193,  0.0353, -0.0249],
        [-0.0145,  0.0350,  0.0248,  ...,  0.0065,  0.0308, -0.0297]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0030, -0.0217], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0243, -0.0277, -0.0412,  ...,  0.0014, -0.0199, -0.0322],
        [ 0.0060,  0.0202, -0.0085,  ..., -0.0146, -0.0064,  0.0316]],
       device='cuda